# 개체명 인식(NER)

In [5]:
# 필요한 라이브러리 설치
!pip install torch

In [ ]:


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
from collections import defaultdict, Counter
import re

# 샘플 데이터 준비 (단어 단위로 분리)
sentences = [
    ["김철수", "는", "서울대학교", "에서", "컴퓨터과학", "을", "전공", "했습니다"],
    ["이영희", "가", "부산", "에", "있는", "회사", "에", "취직", "했어요"],
    ["박민수", "는", "삼성전자", "에서", "일", "합니다"],
    ["최지영", "이", "구글", "코리아", "에", "지원", "했습니다"],
    ["손흥민", "은", "토트넘", "에서", "축구", "를", "합니다"],
    ["정현", "이", "윔블던", "테니스", "대회", "에", "출전", "했다"],
    ["김연아", "는", "피겨스케이팅", "선수", "입니다"],
    ["이순신", "장군", "은", "조선", "시대", "의", "인물", "이다"]
]


# B-PER: 사람 이름의 시작
# I-PER: 사람 이름의 연속
# B-ORG: 조직명의 시작
# O: 일반 단어
# BIO 태깅 레이블
ner_labels = [
    ["B-PER", "O", "B-ORG", "O", "B-FIELD", "O", "O", "O"],
    ["B-PER", "O", "B-LOC", "O", "O", "O", "O", "O", "O"],
    ["B-PER", "O", "B-ORG", "O", "O", "O"],
    ["B-PER", "O", "B-ORG", "I-ORG", "O", "O", "O"],
    ["B-PER", "O", "B-ORG", "O", "B-FIELD", "O", "O"],
    ["B-PER", "O", "B-EVENT", "B-FIELD", "O", "O", "O", "O"],
    ["B-PER", "O", "B-FIELD", "O", "O"],
    ["B-PER", "O", "O", "B-LOC", "O", "O", "O", "O"]
]

# 어휘 사전 구축
def build_vocab(sentences):
    word_to_idx = {"<PAD>": 0, "<UNK>": 1}
    char_to_idx = {"<PAD>": 0, "<UNK>": 1}

    # 단어 어휘 구축
    for sentence in sentences:
        for word in sentence:
            if word not in word_to_idx:
                word_to_idx[word] = len(word_to_idx)
            # 문자 어휘 구축
            for char in word:
                if char not in char_to_idx:
                    char_to_idx[char] = len(char_to_idx)

    return word_to_idx, char_to_idx

word_to_idx, char_to_idx = build_vocab(sentences)

# 레이블 매핑
label_names = ["O", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC", "B-FIELD", "I-FIELD", "B-EVENT", "I-EVENT"]
label_to_idx = {label: i for i, label in enumerate(label_names)}
idx_to_label = {i: label for label, i in label_to_idx.items()}

# BiLSTM 모델 (CRF 없이 단순화)
class BiLSTM_NER(nn.Module):
    def __init__(self, vocab_size, char_vocab_size, tagset_size, embedding_dim=100, hidden_dim=128, char_embedding_dim=25, char_hidden_dim=25):
        super(BiLSTM_NER, self).__init__()

        # 단어 임베딩
        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)

        # 문자 레벨 LSTM
        self.char_embeds = nn.Embedding(char_vocab_size, char_embedding_dim)
        self.char_lstm = nn.LSTM(char_embedding_dim, char_hidden_dim, bidirectional=True, batch_first=True)

        # 단어 레벨 BiLSTM
        self.lstm = nn.LSTM(embedding_dim + char_hidden_dim * 2, hidden_dim // 2,
                           bidirectional=True, batch_first=True)

        # 드롭아웃
        self.dropout = nn.Dropout(0.3)

        # 출력층
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def get_char_features(self, chars):
        # 문자 임베딩
        char_embeds = self.char_embeds(chars)
        char_lstm_out, _ = self.char_lstm(char_embeds)
        # 마지막과 첫번째 hidden state 연결
        char_features = torch.cat([char_lstm_out[:, -1, :char_lstm_out.size(2)//2],
                                  char_lstm_out[:, 0, char_lstm_out.size(2)//2:]], dim=1)
        return char_features

    def forward(self, words, chars):
        # 단어 임베딩
        word_embeds = self.word_embeds(words)

        # 문자 특성 추출
        batch_size, seq_len, word_len = chars.size()
        chars_flat = chars.view(-1, word_len)
        char_features = self.get_char_features(chars_flat)
        char_features = char_features.view(batch_size, seq_len, -1)

        # 단어와 문자 특성 결합
        combined_embeds = torch.cat([word_embeds, char_features], dim=2)
        combined_embeds = self.dropout(combined_embeds)

        # BiLSTM
        lstm_out, _ = self.lstm(combined_embeds)
        lstm_out = self.dropout(lstm_out)

        # 태그 점수
        tag_scores = self.hidden2tag(lstm_out)

        return tag_scores

# 데이터셋 클래스
class NERDataset(Dataset):
    def __init__(self, sentences, labels, word_to_idx, char_to_idx, label_to_idx, max_word_len=10):
        self.sentences = sentences
        self.labels = labels
        self.word_to_idx = word_to_idx
        self.char_to_idx = char_to_idx
        self.label_to_idx = label_to_idx
        self.max_word_len = max_word_len

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        label_seq = self.labels[idx]

        # 단어를 인덱스로 변환
        word_indices = [self.word_to_idx.get(word, self.word_to_idx["<UNK>"]) for word in sentence]

        # 문자를 인덱스로 변환
        char_indices = []
        for word in sentence:
            word_chars = [self.char_to_idx.get(char, self.char_to_idx["<UNK>"]) for char in word]
            # 패딩 또는 자르기
            if len(word_chars) < self.max_word_len:
                word_chars.extend([self.char_to_idx["<PAD>"]] * (self.max_word_len - len(word_chars)))
            else:
                word_chars = word_chars[:self.max_word_len]
            char_indices.append(word_chars)

        # 레이블을 인덱스로 변환
        label_indices = [self.label_to_idx[label] for label in label_seq]

        return {
            'words': torch.LongTensor(word_indices),
            'chars': torch.LongTensor(char_indices),
            'labels': torch.LongTensor(label_indices)
        }

# 패딩 함수
def collate_fn(batch):
    # 최대 길이 찾기
    max_len = max([len(item['words']) for item in batch])

    # 패딩
    words_batch = []
    chars_batch = []
    labels_batch = []
    masks_batch = []

    for item in batch:
        words = item['words']
        chars = item['chars']
        labels = item['labels']

        # 패딩
        pad_len = max_len - len(words)
        if pad_len > 0:
            words = torch.cat([words, torch.zeros(pad_len, dtype=torch.long)])
            chars = torch.cat([chars, torch.zeros(pad_len, chars.size(1), dtype=torch.long)])
            labels = torch.cat([labels, torch.zeros(pad_len, dtype=torch.long)])

        # 마스크 생성 (실제 단어에 대해서만 True)
        mask = torch.cat([torch.ones(len(item['words'])), torch.zeros(pad_len)])

        words_batch.append(words)
        chars_batch.append(chars)
        labels_batch.append(labels)
        masks_batch.append(mask)

    return {
        'words': torch.stack(words_batch),
        'chars': torch.stack(chars_batch),
        'labels': torch.stack(labels_batch),
        'mask': torch.stack(masks_batch).bool()
    }

# 데이터셋 및 데이터로더 생성
dataset = NERDataset(sentences, ner_labels, word_to_idx, char_to_idx, label_to_idx)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)

# 모델 초기화
model = BiLSTM_NER(len(word_to_idx), len(char_to_idx), len(label_to_idx))
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss(ignore_index=0)  # 패딩 토큰 무시

# 훈련
print("훈련 시작...")
for epoch in range(100):
    total_loss = 0
    model.train()

    for batch in dataloader:
        optimizer.zero_grad()

        words = batch['words']
        chars = batch['chars']
        labels = batch['labels']
        mask = batch['mask']

        # Forward pass
        tag_scores = model(words, chars)

        # 마스크된 위치만 고려하여 손실 계산
        active_loss = mask.view(-1) == 1
        active_logits = tag_scores.view(-1, len(label_to_idx))[active_loss]
        active_labels = labels.view(-1)[active_loss]

        loss = criterion(active_logits, active_labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    if epoch % 20 == 0:
        print(f'Epoch {epoch}, Loss: {total_loss:.4f}')

# 예측 함수
def predict_ner(sentence):
    model.eval()

    # 문장 전처리
    if isinstance(sentence, str):
        # 간단한 한국어 토큰화 (실제로는 더 정교한 방법 필요)
        words = re.findall(r'[가-힣]+|[a-zA-Z]+|[0-9]+', sentence)
    else:
        words = sentence

    # 인덱스 변환
    word_indices = [word_to_idx.get(word, word_to_idx["<UNK>"]) for word in words]

    char_indices = []
    for word in words:
        word_chars = [char_to_idx.get(char, char_to_idx["<UNK>"]) for char in word]
        if len(word_chars) < 10:
            word_chars.extend([char_to_idx["<PAD>"]] * (10 - len(word_chars)))
        else:
            word_chars = word_chars[:10]
        char_indices.append(word_chars)

    # 텐서로 변환
    words_tensor = torch.LongTensor(word_indices).unsqueeze(0)
    chars_tensor = torch.LongTensor(char_indices).unsqueeze(0)

    with torch.no_grad():
        tag_scores = model(words_tensor, chars_tensor)
        predictions = torch.argmax(tag_scores, dim=2)

    # 결과 반환
    result = []
    for word, tag_idx in zip(words, predictions[0]):
        result.append((word, idx_to_label[tag_idx.item()]))

    return result

# 정확도 계산 함수
def calculate_accuracy(model, dataloader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in dataloader:
            words = batch['words']
            chars = batch['chars']
            labels = batch['labels']
            mask = batch['mask']

            tag_scores = model(words, chars)
            predictions = torch.argmax(tag_scores, dim=2)

            # 마스크된 위치만 고려
            active_mask = mask.view(-1) == 1
            active_predictions = predictions.view(-1)[active_mask]
            active_labels = labels.view(-1)[active_mask]

            correct += (active_predictions == active_labels).sum().item()
            total += active_labels.size(0)

    return correct / total

# 최종 정확도 출력
accuracy = calculate_accuracy(model, dataloader)
print(f"\n훈련 정확도: {accuracy:.4f}")

# 테스트
print("\n=== NER 예측 결과 ===")
test_sentences = [
    "손흥민은 토트넘에서 축구를 합니다",
    "김연아는 피겨스케이팅 선수입니다",
    "삼성전자에서 스마트폰을 개발합니다",
    "BTS는 전세계적으로 유명한 그룹입니다"
]

for test_sentence in test_sentences:
    result = predict_ner(test_sentence)
    print(f"입력: {test_sentence}")
    print(f"결과: {result}")

    # 개체명만 추출해서 보여주기
    entities = []
    for word, tag in result:
        if tag.startswith('B-'):
            entities.append((word, tag[2:]))
    if entities:
        print(f"추출된 개체명: {entities}")
    print("-" * 50)

훈련 시작...
Epoch 0, Loss: 9.5637
Epoch 20, Loss: 0.0069
Epoch 40, Loss: 0.0023
Epoch 60, Loss: 0.0007
Epoch 80, Loss: 0.0010

훈련 정확도: 0.3448

=== NER 예측 결과 ===
입력: 손흥민은 토트넘에서 축구를 합니다
결과: [('손흥민은', 'B-PER'), ('토트넘에서', 'B-ORG'), ('축구를', 'B-ORG'), ('합니다', 'B-ORG')]
추출된 개체명: [('손흥민은', 'PER'), ('토트넘에서', 'ORG'), ('축구를', 'ORG'), ('합니다', 'ORG')]
--------------------------------------------------
입력: 김연아는 피겨스케이팅 선수입니다
결과: [('김연아는', 'B-PER'), ('피겨스케이팅', 'B-FIELD'), ('선수입니다', 'B-ORG')]
추출된 개체명: [('김연아는', 'PER'), ('피겨스케이팅', 'FIELD'), ('선수입니다', 'ORG')]
--------------------------------------------------
입력: 삼성전자에서 스마트폰을 개발합니다
결과: [('삼성전자에서', 'B-ORG'), ('스마트폰을', 'B-ORG'), ('개발합니다', 'B-ORG')]
추출된 개체명: [('삼성전자에서', 'ORG'), ('스마트폰을', 'ORG'), ('개발합니다', 'ORG')]
--------------------------------------------------
입력: BTS는 전세계적으로 유명한 그룹입니다
결과: [('BTS', 'B-PER'), ('는', 'B-PER'), ('전세계적으로', 'B-ORG'), ('유명한', 'B-ORG'), ('그룹입니다', 'B-ORG')]
추출된 개체명: [('BTS', 'PER'), ('는', 'PER'), ('전세계적으로', 'ORG'), ('유명한', 'ORG'), (